In [ ]:
import numpy as np 
from sklearn import datasets
import matplotlib.pyplot as plt
import os, sys

In [ ]:
import kmapper as km
from kmapper.plotlyviz import *
import sklearn

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_dir = '../data/corr-mats/TimeSeriesAAL/'

fs = os.listdir(data_dir)
ts_data = {}
subs = []
for f in fs:
    sub = f.split('_')[1]
    subs.append(sub)
    test = int(f.split('_')[3][-1]);
    if sub not in ts_data:
        ts_data[sub] = {}
    ts_data[sub][test] = np.loadtxt(data_dir+f)

In [ ]:
kms = {}
for sub in subs:
    X = np.array(ts_data[sub][0])
    # Create another 2-D lens with UMAP
    mapper = km.KeplerMapper(verbose=0)
    import umap
    lens = mapper.fit_transform(X, projection=umap.UMAP(n_neighbors=15,
                                                        min_dist=0.3,
                                                        n_components=2,
                                                        metric='correlation',
                                                        random_state=3571))

    # Define the simplicial complex
    kms[sub] = mapper.map(lens,X,
                          nr_cubes=15,
                          overlap_perc=0.7,
                          clusterer=sklearn.cluster.KMeans(n_clusters=2,
                                                           random_state=3471))



In [ ]:
def mapper_weighting(g):
    import networkx as nx
    gg = g.copy();
    w = {}
    mem_dict = nx.get_node_attributes(g, 'membership')
    for edge in g.edges():
         w[edge] = len(set(mem_dict[edge[0]]).intersection(set(mem_dict[edge[1]])))
    nx.set_edge_attributes(gg,w,'weight');
    return gg;

In [ ]:
gs = {}
wgs = {}
for sub in subs:
    gs[sub] = km.to_networkx(kms[sub]);
    wgs[sub] = mapper_weighting(gs[sub])
    wgs[sub] = nx.relabel_nodes(wgs[sub], dict(zip(wgs[sub].nodes(), range(wgs[sub].number_of_nodes()))))

In [ ]:
def weight_based_filtration(g, min_weight=None):
    import networkx as nx 
    import dionysus as dio
    w = nx.get_edge_attributes(g,'weight');
    ww = [(k, w[k]) for k in sorted(w, key=w.get, reverse=True)]
    if min_weight==None:
        min_weight = ww[-1][1];
    gg = nx.Graph();
    gg.add_nodes_from(g.nodes());
    CD = dio.Filtration()
    index = 0
    for n in g.nodes():
        CD.append(dio.Simplex([n],index))

    for k, v in ww:
        gg.add_edge(k[0],k[1]);
        cliques = nx.find_cliques(gg);
        for clique in cliques:
            if len(clique)>3:
                new_simplices = dio.closure([dio.Simplex(clique)], len(clique));
                for simp in new_simplices:
                    if dio.Simplex(simp) not in CD:
                        CD.append(dio.Simplex(simp,index));
            elif len(clique)==2:
                CD.append(dio.Simplex(clique,index))
        index+=1; 
        if v==min_weight:
            break;
    return CD, index;
    

In [ ]:
fils = {}
for sub in wgs:
    print(sub)
    fils[sub]= weight_based_filtration(wgs[sub],1);

In [ ]:
import dionysus as dio
homs = {};
dgms = {}
for sub in fils:
    homs[sub] = dio.homology_persistence(fils[sub][0])
    dgms[sub] = dio.init_diagrams(homs[sub], fils[sub][0])
    
    

In [ ]:
new_diag = {}


for sub in dgms:
    new_diag[sub] = {}
    dim = 0
    for dgm in dgms[sub]:
        new_diag[sub][dim] = []
        for el in dgm:
            new_el = [el.birth, el.death]
            if new_el[1] == np.inf:
                new_el[1] = fils[sub][1]
        new_diag[sub][dim].append(new_el);
        dim+=1


In [ ]:
import persim

dim = 1
deh = np.zeros((len(dgms), len(dgms)));
for i, f in enumerate(dgms.keys()):
    for j, ff in enumerate(dgms.keys()):
        if len(new_diag[f])>=1 and len(new_diag[ff])>=1:
            deh[i,j] = persim.sliced_wasserstein(np.array(new_diag[f][dim]),np.array(new_diag[ff][dim]),10)


In [ ]:
plt.imshow(deh)
plt.colorbar()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
plt.imshow(deh)